# Equação de Tolman-Oppenheimer-Volkoff (TOV)

Partimos da métrica de Swarzschild (trabalhamos com unidades onde $c = G = 1$):

$$ds^2 =  -\left( 1 - \frac{2M}{r} \right) dt^2 + {\left( 1 - \frac{2M}{r} \right)}^{-1} dr^2 + r^2 d\theta^2 + r^2 \sin^2 \theta d\phi^2$$

e do tensor de energia momentum para um fluido perfeito:

$$T^{\mu \nu} = (\rho + p)U^{\mu} U^{\nu} + p g^{\mu \nu} $$

As equações de campo de Einsten nos dão a relação entre a métrica e o tensor de energia momentum:

$$R_{\mu \nu} - \frac{1}{2} R g_{\mu \nu} = 8 \pi T_{\mu \nu}$$

Com isso podemos igualar ambas essas grandezas e, após um pouco de álgebra, chegamos a:

$$\frac{dp}{dr} = -\frac{m(r) \epsilon (r)}{r^2} \left[ 1 - \frac{p(r)}{\epsilon(r)} \right] \left[ 1 + \frac{4 \pi r^3 p(r)}{m(r)} \right] {\left[ 1 - \frac{2m(r)}{r} \right]}^{-1}$$
$$\frac{dm}{dr} = 4 \pi r^2 \epsilon(r)$$


São duas equações diferenciais acopladas, que, com as condições de que a massa no centro seja $m(0) = 0$ e que a pressão na superfície seja $p(r = R) = 0$ onde $R$ ficará definido como o raio da estrela e $m(r = R) = M$ é a massa total da estrela (massa gravitacional). Ainda precisamos de uma condição inicial na pressão que será $p(0) = p_0$, essa condição porém será arbitrária. *Estrelas com $p_0$ diferente terão valores de massa ($M$) e raio ($R$) diferentes*. Ainda nos falta uma informação, que é $\epsilon(r)$.

# Equação de estado

É a equação de estado que nos diz sobre como a matéria da estrela é constituida e como ela interage. Com uma equação de estado podemos encontrar uma relação entre $p(r)$ e $\epsilon(r)$ e finalmente resolvermos o sistema. A equação de estado mais simples que existe é a equação de estado politrópica onte temos:

$$p = K\epsilon^{\gamma}$$

onde $K$ é uma constante e $\gamma$ um índice politrópico (não lembro ao certo se o $\gamma$ é o índice politrópico). Porém, essa equação de estado simplesmente considera a estrela como uma bola de gás e podemos achar bem difícil e não tão agradável trabalhar com equações de estado politrópicas.

# Resolvendo a TOV

Por termos muitas constantes e grandezas físicas com ordens de grandezas muito diferentes, podemos encontrar problemas numéricos ao tentarmos resolver. Para evitar tais problemas vamos trabalhar com um sistema de unidades onde $c = G = M_{\odot} = 1$ onde $c$ é a velocidade da luz, $G$ é a constante de Newton e $M_{\odot}$ é a massa do Sol (veremos mais tarde por quê é interessante fazermos ela igual a 1 também). Após um desenvolvimento com unidades (presente em outro notebook nesse repositório) encontramos os fatores de conversão das unidades de comprimento, massa e tempo do SI para esse sistema definido.

$$\text{unidade de massa} = M_{\odot} = 1.989 \times 10^{30} kg$$
$$\text{unidade de distancia} = \frac{G M_{\odot}}{c^2} = 1.477 \times 10^3 m$$
$$\text{unidade de tempo} = \frac{G M_{\odot}}{c^3} = 4.927 \times 10^{-6}s$$

O motivo para termos definido a massa do Sol como 1 será para obtermos as massas sempre em massas solares (que vai ser muito útil pois estamos resolvendo uma equação para a massa de uma estrela).

Sabendo esses fatores de conversão podemos facilmente converter todas as grandezas (como aquelas da equação de estado) para esse sistema de unidades. Todo esse trabalho com unidades será localizado em um arquivo com todas as constantes necessárias e fatores de conversão: `constants.jl`. 

Vamos agora incluir os arquivos necessários para resolvermos a TOV aqui:

In [7]:
include("constants.jl")
include("diff.jl")

solve_system

In [8]:
pressure_eq(r, p, m) = -(m*eos(p)/r^2)*(1 - p/eos(p))*(1 + 4πr^(3)*p/m)*(1 - 2m/r)^(-1)
mass_eq(r, p, m) = 4πr^(2)*eos(p)

condition(t, u) = u[1] <= 0 ? false : true

println(pa)

r0 = 1e-8
m0 = 1e-24
p0 = 100MeVfm3

2.8869214805795086e29